# Einflussfaktoren auf die Fahrzeugflotten in deutschen Landkreisen

In [1]:
import os
import pandas as pd

## Einleitung und Daten

### Hintergrund und Motivation
Die Reduktion von Emissionen und die Förderung umweltfreundlicher Fahrzeuge sind zentrale Ziele der Verkehrspolitik. Die Einführung neuer Emissionsvorschriften, wie die Euro6-Norm, sowie die Verbreitung neuer Technologien, wie Elektrofahrzeuge und Plug-in-Hybride, haben das Potenzial, die Zusammensetzung der Fahrzeugflotten in den Landkreisen erheblich zu beeinflussen. Gleichzeitig können sozioökonomische Faktoren wie das verfügbare Einkommen pro Einwohner und die Unfallrate eine Rolle bei der Erneuerung der Fahrzeugflotten spielen.

### These
Die gesetzliche Einführung neuer Emissionsvorschriften wird voraussichtlich einen signifikanten Einfluss auf die Bestandsflotte haben. Es wird erwartet, dass der Anteil von Euro4-Fahrzeugen in Landkreisen mit einem hohen Anteil neuer Technologien (wie Plug-in-Hybriden und Elektrofahrzeugen) abnimmt. Gleichzeitig könnten sozioökonomische Faktoren wie das verfügbare Einkommen und die Unfallrate ebenfalls eine Rolle bei der Erneuerung der Fahrzeugflotten spielen. In Landkreisen mit einer älteren Fahrzeugflotte, die durch einen hohen Anteil von Fahrzeugen der Emissionsgruppen Euro2 und Euro3 gekennzeichnet ist, wird jedoch erwartet, dass der Anteil von Euro4-Fahrzeugen trotz neuer Emissionsvorschriften und Technologien robust bleibt.

### Forschungsfrage und Ziel des Modells
Welche Faktoren beeinflussen den Anteil von Euro4-Fahrzeugen in deutschen Landkreisen und wie stark ist dieser Einfluss?

Das multiple lineare Regressionsmodell soll dazu verwendet werden, die folgenden Fragen zu beantworten:
 - Identifikation relevanter Prädiktoren: Welche Variablen haben einen signifikanten Einfluss auf den Anteil von Euro4-Fahrzeugen?
 - Quantifizierung des Einflusses: Wie stark ist der Einfluss der identifizierten Prädiktoren auf den Anteil von Euro4-Fahrzeugen?
 - Modellgüte und Generalisierbarkeit: Wie gut erklärt das Modell die Varianz im Anteil von Euro4-Fahrzeugen und wie gut generalisiert es auf neue Daten?

### Datenbeschreibung

Für die Analyse soll ein Gesamt-Datensatz aus den folgenden vier Datenquellen erstellt werden:
1. Daten über Fahrzeugbestand
2. Bevölkerungsdaten
3. Daten über Einkommen der privaten Haushalte
4. Straßenverkehrsunfälle

Jede dieser Datenquellen ist nicht für jedes Jahr verfügbar. Eine gemeinsame Schnittmenge existiert für das Jahr 2019, in dem alle Teil-Datensätze erhoben wurden. Daher wird der Gesamt-Datensatz auf das Jahr 2019 beschränkt.

Vorteile der Beschränkung auf das Jahr 2019
- Vermeidung von Zeitabhängigkeiten: Durch die Beschränkung auf ein Jahr werden zeitliche Abhängigkeiten und Trends eliminiert, was die Modellierung vereinfacht.
- Konsistenz der Daten: Alle Datenquellen sind für das Jahr 2019 verfügbar, was eine konsistente und vollständige Datenbasis gewährleistet.
- Fokus auf Querschnittsdaten: Für ein multiples lineares Regressionsmodell kann es sinnvoll sein, sich auf Querschnittsdaten zu konzentrieren, um die Einflüsse der verschiedenen Prädiktoren auf die Zielvariable zu isolieren.

Die Rohdaten für dieses Projekt sind im Projektverzeichnis unter `data/raw` abgelegt. Von dort aus müssen sie zur weiteren Bearbeitung eingelesen werden.

In [2]:
root_raw = os.path.join('..', 'data', 'raw')

data_kfz = '46251-0021_de_2020_flat.csv' # Daten über Fahrzeugbestand
data_pop = '12211-Z-03_flat.csv' # Bevölkerungsdaten
data_vee = 'AI-S-01_flat.csv' # Daten über Einkommen der privaten Haushalte
data_svu = 'AI013-3_flat.csv' # Straßenverkehrsunfälle

In [3]:
df_kfz = pd.read_csv(os.path.join(root_raw, data_kfz), sep=";", decimal='.') # Fahrzeugbestand
df_pop = pd.read_csv(os.path.join(root_raw, data_pop), sep=";", decimal='.', encoding='ISO-8859-1') # Bevölkerungsdaten
df_vee = pd.read_csv(os.path.join(root_raw, data_vee), sep=";", decimal='.', encoding='ISO-8859-1') # Einkommen
df_svu = pd.read_csv(os.path.join(root_raw, data_svu), sep=";", decimal='.', encoding='ISO-8859-1') # Straßenverkehrsunfälle

**1. Daten über Fahrzeugbestand (nach Kraftstoffart und Emissionsgruppen)**
- Quelle: [Statistik des Kraftfahrzeug- und Anhängerbestandes, Statistisches Bundesamt, Code: 46251-0021](https://www-genesis.destatis.de/genesis/online/data)
- Beschreibung der Beobachtungen im Datensatz: Vom Kraftfahrt-Bundesamt werden aus Meldungen der Kraftfahrzeug-Zulassungsstellen Statistiken über den Bestand an Kraftfahrzeugen und Anhängern (jährlich), Neuzulassungen (monatlich), Besitzumschreibungen (monatlich) und Löschungen (jährlich) durchgeführt. Die statistischen Auswertungen umfassen alle nach der Straßenverkehrs-Zulassungs-Ordnung (StVZO) in Deutschland zugelassenen bzw. angemeldeten Fahrzeuge, denen ein amtliches Kennzeichen zugeteilt wurde. Mit einbezogen sind Fahrzeuge, die vorübergehend stillgelegt wurden sowie Fahrzeuge des Bundesgrenzschutzes (BGS) und des Technischen Hilfswerkes (THW). Dagegen sind nicht einbezogen die Fahrzeuge der Bundeswehr sowie Fahrzeuge mit rotem Kennzeichen und mit Ausfuhrkennzeichen.
- Datenerhebung: Dem Kraftfahrtbundesamt (KBA) obliegt die statistische Bearbeitung der Datenmeldungen der Zulassungsbehörden und der Haftpflichtversicherungen (Kfz mit Versicherungs-kennzeichen) sowie des Zentralen Fahrzeugregisters (ZFZR). Es handelt sich dabei um Sekundärstatistiken, denen die gemeldeten Kraftfahrzeuge und Kraftfahrzeuganhänger mit deren Halterangaben zugrunde liegen.
- Variablen:
    - Landkreise (476 Merkmalsausprägungen)
    - Kraftstoffarten (8 Merkmalsausprägungen je Landkreis)
    - Emissionsgruppen (10 Merkmalsausprägungen je Kraftstoffart)
    - Anzahl der Fahrzeuge (je Emissionsgruppe, Kraftstoffart, Landkreis)

**2. Bevölkerungsdaten**
- Quelle: [Bevölkerung am Hauptwohnort nach Altersgruppen und Geschlecht, Regionalstatistik, Code: 12211-Z-03](https://www.regionalstatistik.de/genesis/online/)
- Beschreibung der Beobachtungen im Datensatz: Für die Darstellung von Ergebnissen des Mikrozensus auf Individualebene - ohne Bezug zum Haushalts- und Lebensformkontext - wird die "Bevölkerung am Ort der alleinigen bzw. Hauptwohnung" zu Grunde gelegt. Zur "Bevölkerung" gehören nach diesem Bevölkerungsbegriff alle Personen mit nur einer Wohnung sowie Personen mit mehreren Wohnungen am Ort ihrer Hauptwohnung. Dies ist die vorwiegend genutzte Wohnung einer Person. Aus melderechtlichen Gründen werden Soldatinnen und Soldaten im Grundwehrdienst oder auf Wehrübung der Wohngemeinde vor ihrer Einberufung zugeordnet. Entsprechend wird bei Patientinnen und Patienten in Krankenhäusern sowie bei Personen in Untersuchungshaft verfahren. Berufssoldatinnen und Berufssoldaten, Soldatinnen und Soldaten auf Zeit, Angehörige des Bundesgrenzschutzes und der Bereitschaftspolizei in Gemeinschaftsunterkünften werden ebenso wie die Strafgefangenen sowie alle Dauerinsassen von Gemeinschaftsunterkünften und das in diesen Unterkünften wohnende Personal sowohl in den Gemeinden, die diese Unterkünfte beherbergen, als auch in den Gemeinden, in denen sie evtl. einen weiteren Wohnsitz haben, erfasst. Angehörige der ausländischen Stationierungsstreitkräfte sowie der ausländischen diplomatischen und konsularischen Vertretungen mit ihren Familienangehörigen werden grundsätzlich nicht statistisch ermittelt.
- Datenerhebung: Mikrozensus auf Individualebene aus Meldungen der Einwohnermeldeämter
- Variablen:
    - Landkreise (489 Merkmalsausprägungen; Aggregate: Bundesländer, Deutschland)
    - Altersgruppen (4 Merkmalsausprägungen je Landkreis)
    - Geschlecht (2 Merkmalsausprägungen je Altersgruppe)
    - Anzahl der Personen (je Geschlecht, Altersgruppe, Landkreis)

**3. Daten über Einkommen der privaten Haushalte**
- Quelle: [Verfügbares Einkommen je Einwohner, Regionalstatistik, Code: AI-S-01](https://www.regionalstatistik.de/genesis/online/)
- Beschreibung der Beobachtungen im Datensatz: Verfügbares Einkommen der privaten Haushalte einschließlich privater Organisationen ohne Erwerbszweck je Einwohnerin bzw. Einwohner in EUR. Der Indikator gibt Auskunft über den (monetären) Wohlstand der Bevölkerung und ist als der durchschnittliche Betrag zu verstehen, der je Einwohnerin bzw. Einwohner für Konsum- und Sparzwecke zur Verfügung steht. Für die Berechnung des Indikators „Verfügbares Einkommen der privaten Haushalte einschließlich privater Organisationen ohne Erwerbszweck je Einwohnerin bzw. Einwohner in EUR“ wird der Wert des Verfügbaren Einkommens in jeweiligen Preisen im Jahr durch die Zahl der Einwohnerinnen und Einwohner im Jahresdurchschnitt dividiert. Für die Berechnung des Indikators 'Verfügbares Einkommen der privaten Haushalte einschließlich privater Organisationen ohne Erwerbszweck je Einwohnerin bzw. Einwohner in EUR' wird die jahresdurchschnittliche Zahl der Einwohnerinnen und Einwohner auf Basis des Zensus 2011 herangezogen. Die durchschnittliche Jahresbevölkerung ist das arithmetische Mittel aus Jahresanfangs- und Jahresendbestand. Bei diesen Ergebnissen handelt es sich um Ergebnisse der Revision 2014 der Volkswirtschaftlichen Gesamtrechnungen der Länder. Die VGR-Generalrevision 2014 dient in erster Linie der europaweiten Einführung des neuen Europäischen Systems Volkswirtschaftlicher Gesamtrechnungen (ESVG 2010), welches eine Vielzahl von konzeptionellen Änderungen mit sich bringt. Darüber hinaus wurden sämtliche bisherigen Berechnungen und Ergebnisse überprüft sowie neue Erkenntnisse, methodische Verbesserungen und, soweit möglich, neue Datenquellen in das Rechenwerk integriert.
- Datenerhebung: Der Indikator beruht auf Daten des Arbeitskreises "Volkswirtschaftliche Gesamtrechnungen der Länder". Die Ergebnisse beziehen sich immer auf den zum jeweiligen Veröffentlichungstermin maßgeblichen Berechnungsstand.
- Variablen:
    - Landkreise (489 Merkmalsausprägungen; Aggregate: Bundesländer, Deutschland)
    - Verfügbares Einkommen je Einwohner (je Landkreis)

**4. Straßenverkehrsunfälle**
- Quelle: [Straßenverkehrsunfälle bezogen auf Kfz, Regionalstatistik, Code: AI013-3](https://www.regionalstatistik.de/genesis/online/)
- Beschreibung der Beobachtungen im Datensatz: Der Indikator ist ein Maßstab für die Unfallhäufigkeit im Straßenverkehr. Die Kennziffer hat jedoch den Nachteil, dass nur die am Ort gemeldeten Kfz berücksichtigt werden und ein starker Durchgangsverkehr oder ein hohes Pendleraufkommen das Ergebnis verfälschen. Dem Umfang des Verkehrsaufkommens wird also nicht genügend Rechnung getragen. Für die Berechnung des Indikators 'Straßenverkehrsunfälle je 10.000 Kfz' wird die Zahl der Straßenverkehrsunfälle im Jahr durch die Zahl der Kraftfahrzeuge am 01.01. des Jahres dividiert und mit 10.000 multipliziert.
- Datenerhebung: Der Indikator beruht auf Daten aus der Statistik der Straßenverkehrsunfälle sowie auf Daten der amtlichen Kraftfahrzeugstatistik des Kraftfahrt-Bundesamtes zum Kraftfahrzeugbestand.
- Variablen:
    - Landkreise (489 Merkmalsausprägungen; Aggregate: Bundesländer, Deutschland)
    - Straßenverkehrsunfälle je 10.000 Kfz (je Landkreis)

## Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


## Discussion + Conclusion


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your research question along with statistical arguments supporting your conclusions.
In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.
Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.
Lastly, this section will include ideas for future work.